In [1]:
def find_qa_text(question_id):
    try:
        return qa[qa['question_id'] == question_id].iloc[0].raw_data
    except:
        pass

In [2]:
import pandas as pd
train = pd.read_csv("si630w22-hw3-train.csv")
dev = pd.read_csv("si630w22-hw3-dev.csv")
qa = pd.read_csv("si630w22-hw3-data.csv")
qa["raw_data"] = qa["question_text"] + qa["reply_text"]
# qa["raw_data"] = qa["question_text"]
train["qa_text"] = train["id"].apply(find_qa_text)
train["text"] = train["annotator_id"] * 5 + train["group"] * 5 + train["qa_text"] 
dev["qa_text"] = dev["id"].map(find_qa_text)
dev["text"] = dev["annotator_id"] * 5+ dev["group"] * 5 + dev["qa_text"]
final = pd.read_csv("si630w22-hw3-test.public.csv")
final["qa_text"] = final["id"].map(find_qa_text)
final["text"] = final["annotator_id"] * 5+ final["group"] * 5 + final["qa_text"]

# train["qa_text"] = train["id"].apply(find_qa_text)
# train["text"] = train["qa_text"]
# dev["qa_text"] = dev["id"].map(find_qa_text)
# dev["text"] = dev["qa_text"] 

In [3]:
def minus1(rating):
    return rating - 1.0

In [4]:
# train_labels

In [5]:
train_texts, train_labels = train["text"].tolist(), train["rating"].apply(minus1).tolist()
test_texts, test_labels = dev["text"].tolist(), dev["rating"].apply(minus1).tolist()
final_texts, final_labels = final["text"].tolist(), [0 for i in range(len(final))]

In [6]:
from transformers import AutoTokenizer
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained("microsoft/MiniLM-L12-H384-uncased")
max_length = 512
train_encodings = tokenizer(train_texts, truncation=True, padding="max_length", max_length=max_length)
test_encodings = tokenizer(test_texts, truncation=True, padding="max_length", max_length=max_length)
final_encodings = tokenizer(final_texts, truncation=True, padding="max_length", max_length=max_length)
# small_train_encodings = tokenizer(train_texts[0:100], truncation=True, padding="max_length", max_length=max_length)
# small_test_encodings = tokenizer(test_texts[0:100], truncation=True, padding="max_length", max_length=max_length)

In [7]:
class HelpfulnessDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = HelpfulnessDataset(train_encodings, train_labels)
test_dataset = HelpfulnessDataset(test_encodings, test_labels)
final_dataset = HelpfulnessDataset(final_encodings, final_labels)

In [8]:
# small_train_dataset = HelpfulnessDataset(small_train_encodings, train_labels[0:100])
# small_test_dataset = HelpfulnessDataset(small_test_encodings, test_labels[0:100])

In [9]:
from sklearn.metrics import accuracy_score
import numpy as np

def compute_metrics(pred):
  labels = pred.label_ids
  preds = np.round_(pred.predictions)
#   preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

In [10]:
from torch import nn
from transformers import Trainer


class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = torch.nn.MSELoss()
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [12]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("microsoft/Multilingual-MiniLM-L12-H384", num_labels=1).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/Multilingual-MiniLM-L12-H384 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=6,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.03,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=80,
)

trainer = CustomTrainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,             # evaluation dataset
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/home/fangzhel/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 17845
  Num Epochs = 6
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 6696
/home/fangzhel/.local/lib/python3.8/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Step,Training Loss
80,9.274300
160,4.594100
240,2.028300
320,1.331200
400,1.257300
480,1.299800
560,1.270800
640,1.313900
720,1.318300
800,1.202300


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
/home/fangzhel/.local/lib/python3.8/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
/home/fangzhel/.local/lib/python3.8/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the sam

In [ ]:
trainer.evaluate()

In [ ]:
result = trainer.predict(final_dataset).predictions

In [ ]:
results = np.reshape(result, 3821)

In [ ]:
results = results.tolist()

In [ ]:
final["predicted"] = results

In [ ]:
final_rating = final.drop(["annotator_id","group",'qa_text','text'], axis=1)

In [ ]:
final_rating.head(5)

In [ ]:
rating_mean = final_rating.groupby('id').predicted.mean()

In [ ]:
rating_mean = rating_mean.reset_index()

In [ ]:
rating_mean.to_csv("submission.csv", index=False)

In [ ]:
torch.save(model.state_dict(), "./model1")

In [ ]:
# import torch
# from transformers import AutoModelForSequenceClassification
# device = "cuda:0" if torch.cuda.is_available() else "cpu"
# model = AutoModelForSequenceClassification.from_pretrained("microsoft/Multilingual-MiniLM-L12-H384", num_labels=1)
# model.load_state_dict(torch.load("./model"))
# model.to(device)
# model.eval()

In [ ]:
# import pandas as pd
# qa = pd.read_csv("si630w22-hw3-data.csv")
# qa["raw_data"] = qa["question_text"] + qa["reply_text"]
# final = pd.read_csv("si630w22-hw3-test.public.csv")
# final["qa_text"] = final["id"].map(find_qa_text)
# final["text"] = final["annotator_id"] * 5+ final["group"] * 5 + final["qa_text"]
# final_texts = final["text"].tolist()